In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo
/kaggle/input/lstmmodel/lstm_models5.pth
/kaggle/input/lstmmodel/lstm_models_fixed.pth
/kaggle/input/lstmmodel/lstm_models10.pth
/kaggle/input/lstmmodel/lstm_models1.pth
/kaggle/input/lstmmodel/lstm_model15.pth
/kaggle/input/lstmmodel/lstm_model_100.pth
/kaggle/input/t5embeds/train_ids.npy
/kaggle/input/t5embeds/test_embeds.npy
/kaggle/input/t5embeds/train_embeds.npy
/kaggle/input/t5embeds/test_ids.npy


In [28]:
# load data to obtain top 1500 labels
df = pd.read_csv('/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv', sep='\t')
df.head()
num_of_labels = 1500

# Take value counts in descending order and fetch first 1500 `GO term ID` as labels - [:num_of_labels]
labels = df['term'].value_counts().index[:num_of_labels].tolist()

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
from Bio import SeqIO
from Bio.Seq import Seq

In [31]:
''''
function to create a dictionary from train sequence fasta.
This will help us create a dictionary where each key is a protein id and 
the value is the corresponding protein sequence
'''
def createProteinDict(fasta):
    # Create an empty dictionary to store proteinID: proteinValue
    proteinDict = {}
    for record in SeqIO.parse(fasta, "fasta"):
        proteinId = record.id
        protein_sequence = str(record.seq)
        proteinDict[proteinId] = protein_sequence

    return proteinDict

In [32]:
'''
** function to remove unnatural amino acids that are part of the sequence. **
The rationale behind considering the removal of uncommon or non-standard amino acids as 
negligible is based on the fact that these non-common amino acids constitute only about 
1% of the total amino acid composition.
'''
from Bio.Seq import Seq

def remove_unnatural_amino_acids(sequence):
    seq_obj = Seq(sequence)
    
    # replacement values
    amino_acid_mapping = {
        'J': '',
        'U': '',
        'O': '',
        'B': '',
        'X': '',
        'Z': '',
    }
    
    # removing unnatural amino acids
    for unnatural_aa, natural_aa in amino_acid_mapping.items():
        seq_obj = seq_obj.replace(unnatural_aa, natural_aa)
        
    filtered_sequence = ''.join(aa for aa in seq_obj)
    
    return filtered_sequence

In [33]:
!pip install aaindex

In [34]:
'''Using aaIndex1 package to extract values of important physiochemical properties'''
from aaindex import aaindex1
# ARGP820101 Hydrophobicity index (Argos et al., 1982)
hydrophobicity = aaindex1['ARGP820101'].values
print("hydrophobicity")
print(hydrophobicity)

# HOPT810101  Hydrophilicity value (Hopp-Woods, 1981)
hydrophilicity = aaindex1['HOPT810101'].values
print('hydrophilicity')
print(hydrophilicity)

# KARP850101  Flexibility parameter for no rigid neighbors (Karplus-Schulz, 1985)
flexibility = aaindex1['KARP850101'].values
print('flexibility')
print(flexibility)

# CHAM820101  Polarizability parameter (Charton-Charton, 1982)
ssp = aaindex1['CHAM820101'].values
print("Secondary structure propensity")
print(ssp)

#FAUJ880103  Normalized van der Waals volume
nmv = aaindex1['FAUJ880103'].values
print('Normalized van der Waals volume')
print(nmv)

# ZIMJ680103  Polarity (Zimmerman et al., 1968)
polarity = aaindex1['ZIMJ680103'].values
print('polarity')
print(polarity)

# ZIMJ680104  Isoelectric point (Zimmerman et al., 1968)
isoelectric = aaindex1['ZIMJ680104'].values
print('isoelectric')
print(isoelectric)

# ZIMJ680102  Bulkiness (Zimmerman et al., 1968)
bulkiness = aaindex1['ZIMJ680102'].values
print('bulkiness')
print(bulkiness)

# CHAM820101  Polarizability parameter (Charton-Charton, 1982)
polarizability = aaindex1['CHAM820101'].values
print('polarizability')
print(polarizability)

# JANJ780101  Average accessible surface area (Janin et al., 1978)
surfaceArea = aaindex1['JANJ780101'].values
print('Surface Area')
print(surfaceArea)


# add 5 more properties
# ANDN920101 alpha-CH chemical shifts (Andersen et al., 1992)
chemShift = aaindex1['ANDN920101'].values
print('alpha-CH chemical shifts')
print(chemShift)

# BIGC670101 Residue volume (Bigelow, 1967)
vol = aaindex1['BIGC670101'].values
print('R. Volume')
print(vol)

# DAYM780201 Relative mutability (Dayhoff et al., 1978b)
mutability = aaindex1['DAYM780201'].values
print('mutability')
print(mutability)

# FAUJ880108 Localized electrical effect (Fauchere et al., 1988)
electrical = aaindex1['FAUJ880108'].values
print('electrical')
print(electrical)

# CHAM810101 Steric parameter (Charton, 1981)
steric = aaindex1['CHAM810101'].values
print('steric')
print(steric)

hydrophobicity
{'-': 0, 'A': 0.61, 'C': 1.07, 'D': 0.46, 'E': 0.47, 'F': 2.02, 'G': 0.07, 'H': 0.61, 'I': 2.22, 'K': 1.15, 'L': 1.53, 'M': 1.18, 'N': 0.06, 'P': 1.95, 'Q': 0.0, 'R': 0.6, 'S': 0.05, 'T': 0.05, 'V': 1.32, 'W': 2.65, 'Y': 1.88}
hydrophilicity
{'-': 0, 'A': -0.5, 'C': -1.0, 'D': 3.0, 'E': 3.0, 'F': -2.5, 'G': 0.0, 'H': -0.5, 'I': -1.8, 'K': 3.0, 'L': -1.8, 'M': -1.3, 'N': 0.2, 'P': 0.0, 'Q': 0.2, 'R': 3.0, 'S': 0.3, 'T': -0.4, 'V': -1.5, 'W': -3.4, 'Y': -2.3}
flexibility
{'-': 0, 'A': 1.041, 'C': 0.96, 'D': 1.033, 'E': 1.094, 'F': 0.93, 'G': 1.142, 'H': 0.982, 'I': 1.002, 'K': 1.093, 'L': 0.967, 'M': 0.947, 'N': 1.117, 'P': 1.055, 'Q': 1.165, 'R': 1.038, 'S': 1.169, 'T': 1.073, 'V': 0.982, 'W': 0.925, 'Y': 0.961}
Secondary structure propensity
{'-': 0, 'A': 0.046, 'C': 0.128, 'D': 0.105, 'E': 0.151, 'F': 0.29, 'G': 0.0, 'H': 0.23, 'I': 0.186, 'K': 0.219, 'L': 0.186, 'M': 0.221, 'N': 0.134, 'P': 0.131, 'Q': 0.18, 'R': 0.291, 'S': 0.062, 'T': 0.108, 'V': 0.14, 'W': 0.409, 'Y

In [35]:
# list of input dictionaries
listOfProperties = [hydrophobicity, hydrophilicity, flexibility, ssp, nmv, 
                    polarity, isoelectric, bulkiness, polarizability, surfaceArea,
                   chemShift, vol, mutability, electrical, steric]

# initialize the combined dictionary
combinedDictionary = {key: [] for key in listOfProperties[0]}

# populate the combined dictionary with values from input dictionaries
for key in combinedDictionary:
    for dictionary in listOfProperties:
        combinedDictionary[key].append(float(dictionary[key]))

print(combinedDictionary)

{'-': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'A': [0.61, -0.5, 1.041, 0.046, 1.0, 0.0, 6.0, 11.5, 0.046, 27.8, 4.35, 52.6, 100.0, -0.01, 0.52], 'C': [1.07, -1.0, 0.96, 0.128, 2.43, 1.48, 5.05, 13.46, 0.128, 15.5, 4.65, 68.3, 20.0, 0.12, 0.62], 'D': [0.46, 3.0, 1.033, 0.105, 2.78, 49.7, 2.77, 11.68, 0.105, 60.6, 4.76, 68.4, 106.0, 0.15, 0.76], 'E': [0.47, 3.0, 1.094, 0.151, 3.78, 49.9, 3.22, 13.57, 0.151, 68.2, 4.29, 84.7, 102.0, 0.07, 0.68], 'F': [2.02, -2.5, 0.93, 0.29, 5.89, 0.35, 5.48, 19.8, 0.29, 25.5, 4.66, 113.9, 41.0, 0.03, 0.7], 'G': [0.07, 0.0, 1.142, 0.0, 0.0, 0.0, 5.97, 3.4, 0.0, 24.5, 3.97, 36.3, 49.0, 0.0, 0.0], 'H': [0.61, -0.5, 0.982, 0.23, 4.66, 51.6, 7.59, 13.69, 0.23, 50.7, 4.63, 91.9, 66.0, 0.08, 0.7], 'I': [2.22, -1.8, 1.002, 0.186, 4.0, 0.13, 6.02, 21.4, 0.186, 22.8, 3.95, 102.0, 96.0, -0.01, 1.02], 'K': [1.15, 3.0, 1.093, 0.219, 4.77, 49.5, 9.74, 15.71, 0.219, 103.0, 4.36, 105.1, 56.0, 0.0, 0.68], 'L': [1.53, -1.8, 0.967, 0.18

In [36]:
def encodeProteinSequences(sequences, maxSequenceLength, aaindexDict=combinedDictionary):
    encodedSequences = []

    for sequence in sequences:
        encodedSequence = np.zeros((maxSequenceLength, len(aaindexDict['A'])))
        for i, aminoAcid in enumerate(sequence):
            if aminoAcid in aaindexDict:
                encodedSequence[i, :] = aaindexDict[aminoAcid]
            else:
                print('Found an ambiguous amino acid')
                pass

        encodedSequences.append(encodedSequence)
    return np.array(encodedSequences)

In [37]:
# function to truncate protein sequece
def truncateSeq(sequence):
    return sequence[:1000]

In [38]:
fastaFile = '/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta'

In [39]:
testDict = createProteinDict(fastaFile)

In [40]:
for key, value in testDict.items():
    try:
        updatedValue = remove_unnatural_amino_acids(value)
        testDict[key] = updatedValue
    except Exception as e:
        print(f"Error occurred while updating value for key '{key}': {e}")

In [41]:
testProteinIds = np.load('/kaggle/input/t5embeds/test_ids.npy')
print(testProteinIds.shape)
testDf = pd.DataFrame(testProteinIds, columns=['EntryId'])

(141865,)


In [42]:
def getSequence(x, testDict =testDict):
    return testDict[x]

In [43]:
testDf['sequence'] = testDf['EntryId'].apply(getSequence)
display(testDf)

,EntryId,sequence
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...
...,...,...
141860,P08380,GNCKCDDEGPNVRTAPLTGYVDLGYCNEGWEKCASYYSPIAECCRKKK
141861,C0HK72,RGICLEPKVVGPCKARIRRFYYDSETGKCTPFIYGGCGGNGNNFET...
141862,C0HK73,GSICLEPKVVGPCKAGIRRFYFDSETGKCTLFLYGGCKGNGNNFET...
141863,C0HK74,GSICLEPKVVGPCTAYFPRFYFDSETGKCTPFIYGGCEGNGNNFET...


In [44]:
testDf['sequence'] = testDf['sequence'].apply(truncateSeq)

In [45]:
randomLabel = torch.rand(testDf.shape[0], num_of_labels)

In [46]:
#training dataloader
from torch.utils.data import Dataset, DataLoader

class CustomProteinDataset(Dataset):
    def __init__(self, trainX, trainY, transform=None):
        self.x = trainX
        self.y = trainY
        self.transform = transform
        
    def __Xshape__(self):
        return self.x.shape
    
    def __len__(self):
        return self.x.shape[0]
    
    def __Yshape__(self):
        return self.y.shape
    
    # fix get item code
    def __getitem__(self, idx):
        return {
            "input": self.x[idx],
            "label": self.y[idx]
        }

In [47]:

testproteinDataset = CustomProteinDataset(testDf['sequence'].values, randomLabel)

print(testproteinDataset.__Xshape__())
print(testproteinDataset.__Yshape__())

(141865,)
torch.Size([141865, 1500])


In [48]:
numFeatures = 15
hiddenUnits = 100
numLayers = 3
outputDim = 1500
bias = False
batchFirst = True
dropout = .5
batchSize = 400

In [49]:
from torch.autograd import Variable

class lstmModel(nn.Module):
    def __init__(self):
        super(lstmModel, self).__init__()
        self.batchNorm1d = nn.BatchNorm1d(1000)
        self.lstm = nn.LSTM(numFeatures, hiddenUnits, numLayers, bias, batchFirst,dropout=dropout, bidirectional=False)
        self.activation = nn.ReLU()
        self.linear1 = torch.nn.Linear(hiddenUnits, 200)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 400)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(400, 800)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(800, num_of_labels)
        self.sigmoid = nn.Sigmoid()
    
        
    def forward(self, x):
#         x = x.reshape(x.shape[0], 1, x.shape[1])
#         print(x.shape)
#         print(y.shape)
        h0 = Variable(torch.zeros(numLayers, x.size(0), hiddenUnits)).to(device)
        c0 = Variable(torch.zeros(numLayers, x.size(0), hiddenUnits)).to(device)
        x = self.batchNorm1d(x)
        output, (hn, cn) = self.lstm(x, (h0, c0))
#         print(output.shape)
#         print(output[:, -1, :].shape)
#         y = self.activation(output)
        y = self.linear1(output[:, -1, :])
        #print(out1.shape)
        y = self.activation1(y)
        y = self.linear2(y)
        y = self.activation2(y)
        y = self.linear3(y)
        y = self.activation3(y)
        y = self.linear4(y)
#         print(y.shape)
        y = self.sigmoid(y)
#         print(y.shape)
        
        return y
        

In [51]:
#my Model
modelPath = '/kaggle/input/lstmmodel/lstm_models1.pth'
model = lstmModel().to(device)
chkpt = torch.load(modelPath, map_location=torch.device('cpu'))
model.load_state_dict(chkpt)

<All keys matched successfully>

In [52]:
# get inference from the model
preds = []
MAX_LEN=1000
testDataLoader = DataLoader(testproteinDataset, batch_size=batchSize, shuffle=False)
for i, data in enumerate(testDataLoader, 0):
        with torch.no_grad():
            inputX = data['input']
            testEncode = encodeProteinSequences(inputX, MAX_LEN)
            tensorTrain = torch.tensor(testEncode, dtype = torch.float32).to(device)
            outputs = model.forward(tensorTrain)
#             print(outputs.shape)
#             print(type(outputs))
#             paddingSize = inputX.size(0) - outputs.size(0)
#             if(paddingSize >0):
#                 print('Needs padding')
#             padding = torch.zeros(paddingSize, outputs.size(1))
#             outputs_padded = torch.cat((outputs, padding), dim=0)
            preds.append(outputs)

In [54]:
result = torch.cat(preds, dim=0)
print(result.shape)

torch.Size([141865, 1500])


In [59]:
print(result[0])
print(result[1])
print(result[2])
print(result[3])



tensor([0.5721, 0.5698, 0.7608,  ..., 0.0100, 0.0105, 0.0101])
tensor([0.5760, 0.5728, 0.7670,  ..., 0.0081, 0.0085, 0.0081])
tensor([0.5731, 0.5705, 0.7614,  ..., 0.0095, 0.0100, 0.0096])
tensor([0.5737, 0.5709, 0.7654,  ..., 0.0086, 0.0090, 0.0087])


In [56]:
# creates tsv file with Protein ID, corresponding GO terms and confidence score for each GO term
df_submission = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])
test_protein_ids = list(testDf['EntryId'].values)   
l = []
for k in list(test_protein_ids):
    l += [ k] * result.shape[1]
    
df_submission['Protein Id'] = l
df_submission['GO Term Id'] = labels * result.shape[0]
df_submission['Prediction'] = result.ravel()
df_submission.to_csv("submission.tsv",header=False, index=False, sep="\t")

display(df_submission)

,Protein Id,GO Term Id,Prediction
0,Q9CQV8,GO:0005575,0.572144
1,Q9CQV8,GO:0008150,0.569837
2,Q9CQV8,GO:0110165,0.760818
3,Q9CQV8,GO:0003674,0.379030
4,Q9CQV8,GO:0005622,0.451058
...,...,...,...
212797495,A0A3G2FQK2,GO:0051783,0.011140
212797496,A0A3G2FQK2,GO:0031674,0.011344
212797497,A0A3G2FQK2,GO:0001818,0.010940
212797498,A0A3G2FQK2,GO:0006874,0.011471


In [57]:
df_submission.dtypes

Protein Id     object
GO Term Id     object
Prediction    float32
dtype: object

In [58]:
df_submission.head(50) 

,Protein Id,GO Term Id,Prediction
0,Q9CQV8,GO:0005575,0.572144
1,Q9CQV8,GO:0008150,0.569837
2,Q9CQV8,GO:0110165,0.760818
3,Q9CQV8,GO:0003674,0.379030
4,Q9CQV8,GO:0005622,0.451058
5,Q9CQV8,GO:0009987,0.408367
6,Q9CQV8,GO:0043226,0.420580
7,Q9CQV8,GO:0043229,0.380484
8,Q9CQV8,GO:0005488,0.340050
9,Q9CQV8,GO:0043227,0.253161


The submission.tsv is submitted (on kaggle submission portal) to evaluate the performance of the model.

# References
1. Starter notebook provided by the organizers of CAFA-5
2. https://www.kaggle.com/code/alexandervc/baseline-multilabel-to-multitarget-binary